In [71]:
! pip install --quiet python-dotenv openai tabulate
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [72]:
import evadb
import shutil
import pandas as pd
import os
from dotenv import load_dotenv

In [74]:
shutil.rmtree('./evadb_data')

try:
    cursor = evadb.connect().cursor()
    cursor.query("""
        CREATE DATABASE mydb WITH ENGINE = 'sqlite', PARAMETERS = {
        "database": ":memory:"
    };""").execute()
except:
    pass

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: './evadb_data\\evadb.db'

In [90]:


# drop all tables
cursor.query("""
    DROP TABLE IF EXISTS tasks;
""").execute()
# cursor.query("""
#     DROP TABLE IF EXISTS members;
# """).df()

# Create tables
cursor.query("""
    CREATE TABLE tasks (
        id INTEGER,
        name TEXT(30),
        description	TEXT(100)
    );
""").execute()


In [76]:
# list tables
x = cursor.query("""
use mydb {
   SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%'
};
""").df()

print(x)

Empty DataFrame
Columns: []
Index: []


In [92]:
# create tasks

# y = cursor.query("""
#  INSERT INTO tasks (id, name, description) VALUES (3, 'test name 2', 'test description');
#     """).df()
# print(y)


# drop all data in tasks

# read json using pd
json_data = pd.read_json('issues.json')
print(json_data.head())

# insert each row into table
for index, row in json_data[1:].iterrows():
    # sanitize
    desc = row['description'].replace("'", "")
    desc = desc.replace(';', '')
    desc = desc.replace(',', '')

    cursor.query("""
    INSERT INTO tasks (id, name, description) VALUES ({}, '{}', '{}');
    """.format(index, row['contributor'], desc)).execute()

                                         description contributor
0  For now, if open's pr adds or changes long int...     xzdandy
1  Check if the query plan uses VectorIndexScan f...    jiashenC
2  I'm planning to add a new CONCAT operator to E...  pchunduri6
3  When training models, tuning the parameters ar...     xzdandy
4  Right now, the binder_context is not getting u...   gaurav274


In [93]:
tasks = cursor.table('tasks').select("*").df()
print(tasks.head())

   tasks._row_id  tasks.id  tasks.name  \
0              1         1    jiashenC   
1              2         2  pchunduri6   
2              3         3     xzdandy   
3              4         4   gaurav274   
4              5         5   gaurav274   

                                   tasks.description  
0  Check if the query plan uses VectorIndexScan f...  
1  Im planning to add a new CONCAT operator to Ev...  
2  When training models tuning the parameters are...  
3  Right now the binder_context is not getting up...  
4  Right now for nested SELECT queries the rename...  


c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\models\storage\batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


In [79]:
# print(os.environ['OPENAI_KEY'])

# cursor.query("""
# CREATE FUNCTION IF NOT EXISTS OpenAIChatCompletion
# IMPL 'evadb/functions/openai_chat_completion_function.py'
# MODEL 'gpt-3.5-turbo'
# """).execute()

In [96]:
# sample_issue = "When training models, tuning the parameters are common. Adding a CREATE OR REPLACE to simplify the query, so users do not need to run a DROP query every time."
sample_issue = json_data['description'][0]
# sanitize
sample_issue = sample_issue.replace("'", "")
sample_issue = sample_issue.replace(';', '')
sample_issue = sample_issue.replace(',', '')

# query = f"""
# Give a similarity score between this and the sample task below. Here are all of the tasks as context:
# All tasks:
# {tasks.to_markdown()}
# """

context = tasks.to_markdown()

query = f"""
How similar is the following sample task to the given context task? {sample_issue}
 
all tasks and contributors:

{context}

"""
# query = f"""
# Which of the following tasks is this most similar to?
# All tasks:
# {tasks.to_markdown()}
# """


# query = f"Give only a value between 0 and 1 representing the similarity between the given description and the following sample issue: {sample_issue}"
# prompt = "Give only a value between 0 and 1 for similarity and limit your response to 3 words."
# prompt = "Give only the tasks.id of the most similar task. Limit your response to 3 words."
prompt = "return based on the following [not similar = 0, somewhat similar = 0.5, very similar = 1]. limit your response to 1 number between 0 and 1."
res = cursor.table("tasks").select(
    f"ChatGPT('{query}', description, '{prompt}')"
)
responses = res.df()



print(responses)

c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\models\storage\batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


   chatgpt.response
0               0.1
1               0.1
2               0.2
3               0.2
4               0.1
5               0.1
6               0.1
7               0.2
8               0.1
9               0.2
10              0.1
11              0.5
12              0.3
13              0.2
14              0.2
15              0.1
16              0.1
17              0.1
18              0.1
19              0.1
20              0.2


c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\models\storage\batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


In [97]:
# print all of the responses in responses['chatgpt.response']


#  append this col to tasks
tasks['similarity'] = responses['chatgpt.response']


# sort by similarity
tasks = tasks.sort_values(by=['similarity'], ascending=False)
print(tasks.head())

    tasks._row_id  tasks.id     tasks.name  \
11             12        12        xzdandy   
12             13        13       hershd23   
20             21        21       jiashenC   
14             15        15  Chitti-Ankith   
2               3         3        xzdandy   

                                    tasks.description similarity  
11  We want to enable coverage check for branches ...        0.5  
12  Currently we only test the releases wheel by r...        0.3  
20               Automate the minor release procedure        0.2  
14  SQLite integration: Similar to Postgres add su...        0.2  
2   When training models tuning the parameters are...        0.2  


In [98]:
print(f"""
This task should be assigned to: {tasks.iloc[0]['tasks.name']}
""")


This task should be assigned to: xzdandy

